In [1]:
import tensorflow as tf
import matplotlib as pyplot
import numpy as np
tf.keras.mixed_precision.set_global_policy('mixed_float16')

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 3090, compute capability 8.6


In [2]:
img_size=512

train_set = 'F:/C_D_W/train'
valid_set = 'F:/C_D_W/val'

data_aug = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.Rescaling(scale=1./255,input_shape=(img_size,img_size,3)),
    tf.keras.layers.experimental.preprocessing.RandomZoom(0.2),
    tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
   # tf.keras.layers.experimental.preprocessing.RandomWidth(0.1),
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.1)
])

data_aug_test= tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.Rescaling(scale=1./255)
    ])

In [3]:
train = tf.keras.preprocessing.image_dataset_from_directory(train_set, image_size=(img_size,img_size),batch_size=64)
class_names = train.class_names
num_classes = len(train.class_names)
train = train.map(lambda x,y:(data_aug(x,training=True),y),num_parallel_calls=12).prefetch(tf.data.AUTOTUNE).cache()

Found 14630 files belonging to 3 classes.


In [4]:
valid = tf.keras.preprocessing.image_dataset_from_directory(valid_set, image_size=(img_size,img_size),batch_size=64,subset='training',seed=52,validation_split=0.4)
valid = valid.map(lambda x,y:(data_aug_test(x,training=True),y),num_parallel_calls=12).prefetch(tf.data.AUTOTUNE).cache()

Found 1500 files belonging to 3 classes.
Using 900 files for training.


In [5]:
test = tf.keras.preprocessing.image_dataset_from_directory(valid_set, image_size=(img_size,img_size),batch_size=64,subset='validation',seed=52,validation_split=0.4)
test = test.map(lambda x,y:(data_aug_test(x,training=True),y),num_parallel_calls=12).prefetch(tf.data.AUTOTUNE)

Found 1500 files belonging to 3 classes.
Using 600 files for validation.


In [6]:
transfer_model = tf.keras.applications.Xception(include_top=False)
transfer_model.trainable=False

In [7]:
inputs = tf.keras.layers.Input(shape=(img_size,img_size,3))
x=transfer_model(inputs, training=False)
x=tf.keras.layers.GlobalAveragePooling2D()(x)
x=tf.keras.layers.Dense(num_classes)(x)
outputs = tf.keras.layers.Activation('softmax',dtype=tf.float32,name='activation')(x)
model_1 = tf.keras.Model(inputs,outputs)

In [8]:
model_1.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])

In [9]:
transfer = model_1.fit(train,epochs=3,validation_data=valid)

Epoch 1/3
229/229 [==============================] - 108s 436ms/step - loss: 0.1836 - acc: 0.9772 - val_loss: 0.0421 - val_acc: 0.9989
Epoch 2/3
229/229 [==============================] - 33s 142ms/step - loss: 0.0303 - acc: 0.9969 - val_loss: 0.0192 - val_acc: 0.9989
Epoch 3/3
229/229 [==============================] - 33s 142ms/step - loss: 0.0183 - acc: 0.9973 - val_loss: 0.0122 - val_acc: 0.9989


In [10]:
transfer_model.trainable=True

In [11]:
for layer in transfer_model.layers[:-20]:
    layer.trainable=False

In [12]:
model_1.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])

In [13]:
final_epochs =5
fine_model = model_1.fit(train,epochs=final_epochs,validation_data=valid,initial_epoch=transfer.epoch[-1])

Epoch 3/5
229/229 [==============================] - 43s 175ms/step - loss: 0.4921 - acc: 0.9672 - val_loss: 0.0102 - val_acc: 0.9978
Epoch 4/5
229/229 [==============================] - 39s 171ms/step - loss: 0.0207 - acc: 0.9954 - val_loss: 0.0038 - val_acc: 0.9989
Epoch 5/5
229/229 [==============================] - 40s 173ms/step - loss: 0.0069 - acc: 0.9976 - val_loss: 0.0012 - val_acc: 1.0000


In [14]:
model_1.evaluate(test)

10/10 [==============================] - 2s 137ms/step - loss: 0.0241 - acc: 0.9967


[0.024145158007740974, 0.996666669845581]

In [16]:
image = 'C:/Users/akghost96/Downloads/IMG_20210803_120856.jpg'
img = tf.keras.preprocessing.image.load_img(image, target_size=(img_size,img_size))
img = tf.keras.preprocessing.image.img_to_array(img)
img = tf.expand_dims(img,axis=0)
img = img/255

prediction = model_1.predict(img)
class_names[np.argmax(prediction)]


C:\Users\akghost96\AppData\Local\Programs\Python\Python39\lib\site-packages\tensorflow\python\keras\engine\sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


array([1], dtype=int64)

In [ ]:
tf.keras.models.save_model(model_1,'xception_9988.h5')

In [ ]:
model_2 = tf.keras.models.load_model('F:/C_D_W/xception_9988.h5')

In [ ]:
class_names[np.argmax(model_2.predict(img))]